In [2]:
import os
import json
import math
import numpy as np
import pandas as pd

In [3]:
# Folder yang berisi gambar dan file JSON yang berisi label
image_train_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train'
image_test_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid' 
json_train_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train/annotations_with_color_labels1.json'  
json_test_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid/annotations_with_color_labels2.json'

In [4]:
# Fungsi untuk memuat data gambar dan label dari file JSON
def load_data_from_json_and_images(image_folder, json_file):
    images = []
    labels = []
    
    # Membaca file JSON yang berisi anotasi
    with open(json_file) as f:
        data = json.load(f)

    # Membaca gambar dan label dari JSON
    for annotation in data['annotations']:
        image_filename = data['images'][annotation['image_id']]['file_name']  # Ambil nama file gambar berdasarkan image_id
        label = annotation['color_label']  # Label yang berhubungan dengan gambar
        
        # Memuat gambar dari folder
        image_path = os.path.join(image_folder, image_filename)
        image = io.imread(image_path)
        
        # Ubah ukuran gambar jika perlu (agar ukuran konsisten)
        image_resized = resize(image, (64, 64))  # Mengubah ukuran gambar menjadi 64x64
        
        # Flatten gambar menjadi array 1D
        image_flattened = image_resized.flatten()
        
        images.append(image_flattened)
        labels.append(label)
    
    return np.array(images), labels

In [ ]:
def euclidean_distance(p1, p2):
    """
    Menghitung Euclidean Distance antara dua titik RGB.
    p1, p2: Daftar atau tuple dengan nilai RGB, misalnya [R, G, B].
    """
    return math.sqrt(sum((p1[i] - p2[i])**2 for i in range(len(p1))))
